In [0]:
# from pyspark.sql import SparkSession
# from pyspark.sql.functions import col, when, lower, regexp_extract, concat, lit, split, from_json
# from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
# from pyspark.sql.types import StructType, StructField, StringType, TimestampType
# from pathlib import Path
# import json
# from pathlib import Path
# import requests

In [0]:
# #Configuración de conexión JDBC
# JDBC_CONFIG = {
#     "hostname": "psql-dn-keycloak-restore.postgres.database.azure.com",
#     "port": 5432,
#     "database": "keycloak",
#     "username": dbutils.secrets.get(scope='secret-storeview', key='username-keycloak-db'),
#     "password": dbutils.secrets.get(scope='secret-storeview', key='password-keycloak-db'),
#     "driver": "org.postgresql.Driver"
# }

# jdbcUrl = f"jdbc:postgresql://{JDBC_CONFIG['hostname']}:{JDBC_CONFIG['port']}/{JDBC_CONFIG['database']}?sslmode=require"
# connectionProperties = {
#     "user": JDBC_CONFIG["username"],
#     "password": JDBC_CONFIG["password"],
#     "driver": JDBC_CONFIG["driver"]
# }

Transformación

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, TimestampType
from pyspark.sql.functions import col, regexp_extract, from_json, concat, lit


# Definición de esquema
schema = StructType([
    StructField("Activity", StringType(), True),
    StructField("Email", StringType(), True),
    StructField("ItemId", StringType(), True),
    StructField("Item", StringType(), True),
    StructField("Time", TimestampType(), True)
])


# Lectura desde tabla
df_raw = spark.sql("select * from bronze.carga_json_keycloak")

# Procesamiento
df = (
    df_raw
    .filter("resultDescription like '%Activity%'")
    .withColumn(
        "resultDescription_listado",
        concat(
            lit("{"),
            regexp_extract(col("resultDescription"), "\\{(.*?)\\}", 1),
            lit(',"Time":"'), col("time"), lit('"}')
        )
    )
    .select(from_json(col("resultDescription_listado"), schema).alias("parsed"))
    .select("parsed.*")
    .filter(col("Activity") == "View Report")
    .distinct()
)


df.limit(100).display()


In [0]:
# Guardar en delta

path_delta = "/mnt/silver/delta/transform_keycloak"
df.write.format("delta").mode("append").save(path_delta)
spark.sql(f"CREATE TABLE IF NOT EXISTS silver.transform_keycloak USING DELTA LOCATION '{path_delta}/'")
spark.sql("REFRESH TABLE silver.transform_keycloak")

In [0]:
consulta = spark.sql("""select * from silver.transform_keycloak""")
display(consulta)